<h1> Analyze Daily Treasury Statements of Departments that Withdraw & Deposit Cash (by Millions) </h1>

In [1]:
#import modules
import numpy as np
import pandas as pd
import csv

<h3> EXAMINING THE FILE </h3>

In [2]:
#Examine file
initial_file = "Daily_Fed_Budget.csv"
cleaned_file = "Cleaned_Daily_Fed_Budget.csv"

#open files to read from and write to
file = open(initial_file, 'r')
reader = csv.reader(file)

output_file = open(cleaned_file, 'w', newline='')
writer = csv.writer(output_file)

line_number = 1  # Start counting from line 1
removed_rows = []

for row in reader:
    if len(row) == 16:  # Only write rows with exactly 16 fields
        writer.writerow(row)
    else:
        removed_rows.append(row)
        print(f"Line {line_number} has {len(row)} fields and was skipped.")
    
    line_number += 1  # Manually increment the line number
print(removed_rows)

Line 41 has 18 fields and was skipped.
[['2025-01-24', 'Treasury General Account (TGA)', 'Deposits', 'Taxes - IRS Collected Estate', ' Gift', ' misc', '39', '285', '1182', 'II', 'Deposits and Withdrawals of Operating Cash', '39', '2025', '2', '2025', '1', '01', '24']]


In [3]:
# import csv file of Deposits and Withdrawals of Operating Cash
file = "Cleaned_Daily_Fed_Budget.csv"
df = pd.read_csv(file)

<h3> Examine Dataset </h3>

In [5]:
df.head(1)

,Record Date,Type of Account,Transaction Type,Transaction Category,Transactions Today,Transactions Month to Date,Transactions Fiscal Year to Date,Table Number,Table Name,Source Line Number,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number
0,2025-01-24,Treasury General Account (TGA),Withdrawals,NASA,35,1109,7452,II,Deposits and Withdrawals of Operating Cash,167,2025,2,2025,1,1,24


In [6]:
#I want to view the max rows always
pd.set_option("display.max_rows", None)

In [7]:
# Broad examination 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Record Date                       144 non-null    object
 1   Type of Account                   144 non-null    object
 2   Transaction Type                  144 non-null    object
 3   Transaction Category              143 non-null    object
 4   Transactions Today                144 non-null    int64 
 5   Transactions Month to Date        144 non-null    int64 
 6   Transactions Fiscal Year to Date  144 non-null    int64 
 7   Table Number                      144 non-null    object
 8   Table Name                        144 non-null    object
 9   Source Line Number                144 non-null    int64 
 10  Fiscal Year                       144 non-null    int64 
 11  Fiscal Quarter Number             144 non-null    int64 
 12  Calendar Year         

In [8]:
# Check that the entires all have the same date
df['Record Date'].value_counts()

Record Date
2025-01-24    144
Name: count, dtype: int64

In [9]:
# Check all types of Departments
df['Transaction Category'].value_counts() 

# We have 180 departments that withdraw/deposit 

Transaction Category
TREAS - United States Mint                  2
DOL - Pension Benefit Guaranty Corp.        2
HUD - Federal Housing Admin (FHA)           2
Dept of Interior (DOI) - misc               2
DOI - Fish and Wildlife and Parks           2
DOI - Land and Minerals Management          2
Dept of Justice (DOJ)                       2
Dept of Labor (DOL) - misc                  2
Dept of State (DOS)                         2
DHS - Transportation Security Admn (TSA)    2
Dept of Treasury (TREAS) - misc             2
TREAS - Bureau of Engraving and Printing    2
TREAS - Comptroller of the Currency         2
TREAS - Federal Financing Bank              2
Dept of Agriculture (USDA) - misc           2
Dept of Veterans Affairs (VA)               2
Dept of Housing & Urban Dev (HUD) - misc    2
DOI - Water and Science                     2
Dept of Energy (DOE)                        2
DoD - Health                                2
USDA - Federal Crop Insurance Corp Fund     2
Dept of Homel

<h1> Withdrawals </h1>

In [10]:
# How frequently do we withdraw or deposit?
df['Transaction Type'].value_counts() 

# We withdraw more than we deposit

Transaction Type
Deposits       79
Withdrawals    65
Name: count, dtype: int64

In [11]:
# How much do we withdraw, daily, monthly, fiscal ytd?

#Grab sections of the df that align with withdrawals
withdrawals_df = df[ (df['Transaction Type'] == 'Withdrawals')  &  (df['Type of Account'] == 'Treasury General Account (TGA)')]
withdrawals_df["Transaction Type"].value_counts()

Transaction Type
Withdrawals    65
Name: count, dtype: int64

In [12]:
#**********************************************FISCAL YTD*******************************************************

# Get total amount in $ of transactions for fiscal ytd column
ytd_withdrawals = withdrawals_df['Transactions Fiscal Year to Date'].sum()
print('Total sum of transactions fiscal ytd: $' , ytd_withdrawals)

# Show the breakdown per department. In percentages, which department withdraws the most
( (withdrawals_df.groupby('Transaction Category')['Transactions Fiscal Year to Date'].sum()  /  ytd_withdrawals ) *100).sort_values(ascending=False).head(5)


Total sum of transactions fiscal ytd: $ 1399652


Transaction Category
HHS - Grants to States for Medicaid         14.283693
HHS - Federal Supple Med Insr Trust Fund    12.611992
Dept of Defense (DoD) - misc                10.876775
HHS - Federal Hospital Insr Trust Fund       9.441275
Dept of Education (ED)                       5.395698
Name: Transactions Fiscal Year to Date, dtype: float64

In [13]:
#**********************************************Month to Date*******************************************************

# Get total amount in $ of transactions for moth to date column
mtd_withdrawals = withdrawals_df['Transactions Month to Date'].sum()
print('Total sum of transactions month to date: $' , mtd_withdrawals)

# Show the breakdown per department. In percentages, which department withdraws the most
( (withdrawals_df.groupby('Transaction Category')['Transactions Month to Date'].sum()  /  mtd_withdrawals ) * 100).sort_values(ascending=False).head(5)


Total sum of transactions month to date: $ 281232


Transaction Category
HHS - Grants to States for Medicaid         14.697830
HHS - Federal Supple Med Insr Trust Fund    13.905245
HHS - Federal Hospital Insr Trust Fund      10.534363
Dept of Education (ED)                      10.099846
Dept of Defense (DoD) - misc                 9.792271
Name: Transactions Month to Date, dtype: float64

In [14]:
#**********************************************Daily*******************************************************

# Get total amount in $ of transactions for month to date column
today_withdrawals = withdrawals_df['Transactions Today'].sum()
print('Total sum of transactions today: $' , today_withdrawals)

# Show the breakdown per department. In percentages, which department withdraws the most
( (withdrawals_df.groupby('Transaction Category')['Transactions Today'].sum()  /  today_withdrawals ) * 100).sort_values(ascending=False).head(5)


Total sum of transactions today: $ 11671


Transaction Category
Dept of Defense (DoD) - misc                20.272470
Dept of Education (ED)                      14.060492
HHS - Federal Hospital Insr Trust Fund       9.279411
HHS - Grants to States for Medicaid          7.737126
HHS - Federal Supple Med Insr Trust Fund     7.600034
Name: Transactions Today, dtype: float64

<h1> Deposits </h1>

In [15]:
df['Transaction Type'].value_counts()

Transaction Type
Deposits       79
Withdrawals    65
Name: count, dtype: int64

In [16]:
# DF for deposits
deposits_df = df[ (df['Transaction Type']=='Deposits')  &  (df['Type of Account']=='Treasury General Account (TGA)') ]
deposits_df['Transaction Type'].value_counts()

Transaction Type
Deposits    78
Name: count, dtype: int64

In [17]:
#**********************************************FISCAL YTD*******************************************************
ytd_deposits = deposits_df['Transactions Fiscal Year to Date'].sum()
print("Total sum of ytd deposits = $", ytd_deposits)

( ( deposits_df.groupby('Transaction Category')['Transactions Fiscal Year to Date'].sum() / ytd_deposits ) * 100).sort_values(ascending=False).head(5)


Total sum of ytd deposits = $ 11737113


Transaction Category
Public Debt Cash Issues (Table IIIB)        85.506521
Taxes - Withheld Individual/FICA             9.228010
Taxes - Corporate Income                     1.150777
Taxes - Non Withheld Ind/SECA Electronic     1.088794
Taxes - Non Withheld Ind/SECA Other          0.728723
Name: Transactions Fiscal Year to Date, dtype: float64

In [18]:
#**********************************************Month to date*******************************************************
mtd_deposits = deposits_df['Transactions Month to Date'].sum()
print("Total sum of ytd deposits = $", mtd_deposits)

( ( deposits_df.groupby('Transaction Category')['Transactions Month to Date'].sum() / mtd_deposits ) * 100).sort_values(ascending=False).head(5)

Total sum of ytd deposits = $ 2435615


Transaction Category
Public Debt Cash Issues (Table IIIB)        82.283284
Taxes - Withheld Individual/FICA             9.773753
Taxes - Non Withheld Ind/SECA Electronic     3.129559
Taxes - Non Withheld Ind/SECA Other          2.011689
Taxes - Corporate Income                     0.848081
Name: Transactions Month to Date, dtype: float64

In [19]:
#**********************************************Daily*******************************************************
daily_deposits = deposits_df['Transactions Today'].sum()
print("Total sum of ytd deposits = $", daily_deposits)

( ( deposits_df.groupby('Transaction Category')['Transactions Today'].sum() / daily_deposits ) * 100).sort_values(ascending=False).head(5)

Total sum of ytd deposits = $ 25283


Transaction Category
Taxes - Non Withheld Ind/SECA Other         47.423170
Taxes - Withheld Individual/FICA            33.030099
Public Debt Cash Issues (Table IIIB)         6.545900
Taxes - Non Withheld Ind/SECA Electronic     2.195151
Taxes - Corporate Income                     1.566270
Name: Transactions Today, dtype: float64

In [20]:
# FOUND discrepancies between actual sum of deposits and reported sum of deposits
reported_dytd_sum = df[df['Type of Account']== 'Treasury General Account Total Deposits']['Transactions Fiscal Year to Date'].sum()

print("Reported deposit sum fytd: $", reported_dytd_sum)
print("What I found for deposit sum fytd: $", ytd_deposits)
print("Discrepancy: ", (reported_dytd_sum - ytd_deposits), '\n')

reported_dmtd_sum = df[df['Type of Account']== 'Treasury General Account Total Deposits']['Transactions Month to Date'].sum()

print("Reported deposit sum mtd: $", reported_dmtd_sum)
print("What I found for deposit sum mtd: $", mtd_deposits)
print("Discrepancy: ", (reported_dmtd_sum - mtd_deposits), '\n')

reported_dday_sum = df[df['Type of Account']== 'Treasury General Account Total Deposits']['Transactions Today'].sum()

print("Reported deposit sum day: $", reported_dday_sum)
print("What I found for deposit sum day: $",daily_deposits)
print("Discrepancy: ", (reported_dday_sum - daily_deposits))

Reported deposit sum fytd: $ 11738297
What I found for deposit sum fytd: $ 11737113
Discrepancy:  1184 

Reported deposit sum mtd: $ 2435898
What I found for deposit sum mtd: $ 2435615
Discrepancy:  283 

Reported deposit sum day: $ 25320
What I found for deposit sum day: $ 25283
Discrepancy:  37


They deposited less than they said they did.

In [21]:
# FOUND discrepancies between actual sum of withdrawals
reported_ytd_sum = df[df['Type of Account']== 'Treasury General Account Total Withdrawals']['Transactions Fiscal Year to Date'].sum()

print("Reported withdrawals sum fytd: $", reported_ytd_sum)
print("What I found for withdrawals sum fytd: $", ytd_withdrawals)
print("Discrepancy: ", (reported_ytd_sum - ytd_withdrawals), '\n')

reported_mtd_sum = df[df['Type of Account']== 'Treasury General Account Total Withdrawals']['Transactions Month to Date'].sum()

print("Reported withdrawals sum mtd: $", reported_mtd_sum)
print("What I found for withdrawals sum mtd: $", mtd_withdrawals)
print("Discrepancy: ", (reported_mtd_sum - mtd_withdrawals), '\n')

reported_day_sum = df[df['Type of Account']== 'Treasury General Account Total Withdrawals']['Transactions Today'].sum()

print("Reported withdrawals sum day: $", reported_day_sum)
print("What I found for withdrawals sum day: $",today_withdrawals)
print("Discrepancy: ", (reported_day_sum - today_withdrawals))

Reported withdrawals sum fytd: $ 0
What I found for withdrawals sum fytd: $ 1399652
Discrepancy:  -1399652 

Reported withdrawals sum mtd: $ 0
What I found for withdrawals sum mtd: $ 281232
Discrepancy:  -281232 

Reported withdrawals sum day: $ 0
What I found for withdrawals sum day: $ 11671
Discrepancy:  -11671


They withdrew $1 more than they said they did.

In [22]:
# Now lets look at that row we got rid of earlier:
print(removed_rows)

ddf = pd.DataFrame(removed_rows)
ddf


[['2025-01-24', 'Treasury General Account (TGA)', 'Deposits', 'Taxes - IRS Collected Estate', ' Gift', ' misc', '39', '285', '1182', 'II', 'Deposits and Withdrawals of Operating Cash', '39', '2025', '2', '2025', '1', '01', '24']]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,2025-01-24,Treasury General Account (TGA),Deposits,Taxes - IRS Collected Estate,Gift,misc,39,285,1182,II,Deposits and Withdrawals of Operating Cash,39,2025,2,2025,1,01,24


In [23]:
df.head(1)

,Record Date,Type of Account,Transaction Type,Transaction Category,Transactions Today,Transactions Month to Date,Transactions Fiscal Year to Date,Table Number,Table Name,Source Line Number,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number
0,2025-01-24,Treasury General Account (TGA),Withdrawals,NASA,35,1109,7452,II,Deposits and Withdrawals of Operating Cash,167,2025,2,2025,1,1,24


In [24]:
#It seems like the rows nearly match up, the only discrepancy is column 4 and 5. Let's drop those and readd the row to the original dataframr

ddf = ddf.drop(columns=[4,5])
ddf

,0,1,2,3,6,7,8,9,10,11,12,13,14,15,16,17
0,2025-01-24,Treasury General Account (TGA),Deposits,Taxes - IRS Collected Estate,39,285,1182,II,Deposits and Withdrawals of Operating Cash,39,2025,2,2025,1,01,24


In [25]:
# assign the column names of df to ddf
ddf.columns = df.columns

# combine the dataframes vertically (stacked)
df_combined = pd.concat([df, ddf], axis=0, ignore_index=True)
df_combined.tail()

,Record Date,Type of Account,Transaction Type,Transaction Category,Transactions Today,Transactions Month to Date,Transactions Fiscal Year to Date,Table Number,Table Name,Source Line Number,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number
140,2025-01-24,Treasury General Account (TGA),Withdrawals,Taxes - Individual Tax Refunds (EFT),3,140,20562,II,Deposits and Withdrawals of Operating Cash,142,2025,2,2025,1,1,24
141,2025-01-24,Treasury General Account (TGA),Withdrawals,ESF - Economic Recovery Programs,0,50,-1143,II,Deposits and Withdrawals of Operating Cash,143,2025,2,2025,1,1,24
142,2025-01-24,Treasury General Account (TGA),Withdrawals,Dept of Veterans Affairs (VA),283,6666,33401,II,Deposits and Withdrawals of Operating Cash,144,2025,2,2025,1,1,24
143,2025-01-24,Treasury General Account (TGA),Withdrawals,VA - Benefits,155,1268,64823,II,Deposits and Withdrawals of Operating Cash,145,2025,2,2025,1,1,24
144,2025-01-24,Treasury General Account (TGA),Deposits,Taxes - IRS Collected Estate,39,285,1182,II,Deposits and Withdrawals of Operating Cash,39,2025,2,2025,1,01,24


In [26]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Record Date                       145 non-null    object
 1   Type of Account                   145 non-null    object
 2   Transaction Type                  145 non-null    object
 3   Transaction Category              144 non-null    object
 4   Transactions Today                145 non-null    object
 5   Transactions Month to Date        145 non-null    object
 6   Transactions Fiscal Year to Date  145 non-null    object
 7   Table Number                      145 non-null    object
 8   Table Name                        145 non-null    object
 9   Source Line Number                145 non-null    object
 10  Fiscal Year                       145 non-null    object
 11  Fiscal Quarter Number             145 non-null    object
 12  Calendar Year         

In [27]:
# NOW, we should fix that discrepancy we sae



df_combined['Transactions Fiscal Year to Date'] = df_combined['Transactions Fiscal Year to Date'].astype(int)

df_combined['Transactions Month to Date'] = df_combined['Transactions Month to Date'].astype(int)


df_combined['Transactions Today'] = df_combined['Transactions Today'].astype(int)
new_deposits_df = df_combined[ (df_combined['Transaction Type']=='Deposits')  &  (df_combined['Type of Account']=='Treasury General Account (TGA)') ]

new_ytd_deposits = new_deposits_df['Transactions Fiscal Year to Date'].sum()
new_mtd_deposits = new_deposits_df['Transactions Month to Date'].sum()
new_daily_deposits = new_deposits_df['Transactions Today'].sum()


In [28]:
reported_dytd_sum = df[df['Type of Account']== 'Treasury General Account Total Deposits']['Transactions Fiscal Year to Date'].sum()

print("Reported deposit sum fytd: $", reported_dytd_sum)
print("NEW What I found for deposit sum fytd: $", new_ytd_deposits)
print("Discrepancy: ", (reported_dytd_sum - new_ytd_deposits), '\n')

reported_dmtd_sum = df[df['Type of Account']== 'Treasury General Account Total Deposits']['Transactions Month to Date'].sum()

print("Reported deposit sum mtd: $", reported_dmtd_sum)
print("NEW What I found for deposit sum mtd: $", new_mtd_deposits) 
print("Discrepancy: ", (reported_dmtd_sum - new_mtd_deposits), '\n')

reported_dday_sum = df[df['Type of Account']== 'Treasury General Account Total Deposits']['Transactions Today'].sum()

print("Reported deposit sum day: $", reported_dday_sum)
print("NEW What I found for deposit sum day: $",new_daily_deposits )
print("Discrepancy: ", (reported_dday_sum - new_daily_deposits))

Reported deposit sum fytd: $ 11738297
NEW What I found for deposit sum fytd: $ 11738295
Discrepancy:  2 

Reported deposit sum mtd: $ 2435898
NEW What I found for deposit sum mtd: $ 2435900
Discrepancy:  -2 

Reported deposit sum day: $ 25320
NEW What I found for deposit sum day: $ 25322
Discrepancy:  -2


In [29]:
df['Type of Account'].value_counts()

Type of Account
Treasury General Account (TGA)             143
Treasury General Account Total Deposits      1
Name: count, dtype: int64

In [30]:
df[df['Type of Account']== 'Treasury General Account (TGA)']

df[df['Type of Account']== 'Treasury General Account Total Deposits']
#df[df['Type of Account']== 'Treasury General Account Total Withdrawals']


,Record Date,Type of Account,Transaction Type,Transaction Category,Transactions Today,Transactions Month to Date,Transactions Fiscal Year to Date,Table Number,Table Name,Source Line Number,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number
79,2025-01-24,Treasury General Account Total Deposits,Deposits,NaN,25320,2435898,11738297,II,Deposits and Withdrawals of Operating Cash,81,2025,2,2025,1,1,24


In [31]:
new_output_f = "Final_Fed_Budget_Cleaned.csv"
df_combined.to_csv(new_output_f, index=False)


In [32]:
df.columns = [name.replace(" ", "_") for name in df.columns]

df.columns

Index(['Record_Date', 'Type_of_Account', 'Transaction_Type',
       'Transaction_Category', 'Transactions_Today',
       'Transactions_Month_to_Date', 'Transactions_Fiscal_Year_to_Date',
       'Table_Number', 'Table_Name', 'Source_Line_Number', 'Fiscal_Year',
       'Fiscal_Quarter_Number', 'Calendar_Year', 'Calendar_Quarter_Number',
       'Calendar_Month_Number', 'Calendar_Day_Number'],
      dtype='object')

In [33]:
my_df = df_combined
my_df.head(4)

,Record Date,Type of Account,Transaction Type,Transaction Category,Transactions Today,Transactions Month to Date,Transactions Fiscal Year to Date,Table Number,Table Name,Source Line Number,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number
0,2025-01-24,Treasury General Account (TGA),Withdrawals,NASA,35,1109,7452,II,Deposits and Withdrawals of Operating Cash,167,2025,2,2025,1,1,24
1,2025-01-24,Treasury General Account (TGA),Deposits,Dept of Agriculture (USDA) - misc,19,270,1202,II,Deposits and Withdrawals of Operating Cash,1,2025,2,2025,1,1,24
2,2025-01-24,Treasury General Account (TGA),Deposits,USDA - Commodity Credit Corporation,47,573,2147,II,Deposits and Withdrawals of Operating Cash,2,2025,2,2025,1,1,24
3,2025-01-24,Treasury General Account (TGA),Deposits,USDA - Federal Crop Insurance Corp Fund,0,1,18,II,Deposits and Withdrawals of Operating Cash,3,2025,2,2025,1,1,24


In [34]:
# Find departments that deposit the most, and departments that withdraw the most
withd_df = my_df[ (my_df['Transaction Type'] == 'Withdrawals')  &  ((my_df['Type of Account'] == 'Treasury General Account (TGA)'))]
withd_df

withd_total = withd_df['Transactions Fiscal Year to Date'].sum()

temp_w = ( (withd_df.groupby('Transaction Category')['Transactions Fiscal Year to Date'].sum() / withd_total ) * 100).sort_values(ascending=False)

In [58]:
temp_w

Transaction Category
HHS - Grants to States for Medicaid         14.283693
HHS - Federal Supple Med Insr Trust Fund    12.611992
Dept of Defense (DoD) - misc                10.876775
HHS - Federal Hospital Insr Trust Fund       9.441275
Dept of Education (ED)                       5.395698
VA - Benefits                                4.631366
HHS - Medicare Prescription Drugs            3.456073
HHS - Marketplace Payments                   2.950448
USDA - Supp Nutrition Assist Prog (SNAP)     2.530272
DoD - Military Active Duty Pay               2.408956
Dept of Veterans Affairs (VA)                2.386379
TREAS - Federal Financing Bank               2.299357
Dept of Housing & Urban Dev (HUD) - misc     1.963059
DoD - Military Retirement                    1.678203
DHS - Fed Emergency Mgmt Agency (FEMA)       1.535453
HHS - Othr Cent Medicare & Medicaid Serv     1.489799
DOT - Federal Highway Administration         1.470294
Taxes - Individual Tax Refunds (EFT)         1.469079
Dept of

In [36]:
dep_df = my_df[ (my_df['Transaction Type'] == 'Deposits')  &  ((my_df['Type of Account'] == 'Treasury General Account (TGA)'))]
dep_df

dep_total = dep_df['Transactions Fiscal Year to Date'].sum()

temp_d = ((dep_df.groupby('Transaction Category')['Transactions Fiscal Year to Date'].sum() / dep_total) * 100).sort_values(ascending=False)
temp_d.shape

(79,)

In [42]:
df_w = pd.DataFrame(temp_w)
df_w.columns = [['Withdrawal Percentage by Fiscal YTD']]
df_w

df_d = pd.DataFrame(temp_d)
df_d.columns = [['Deposit Percentage by Fiscal YTD']]
df_d

a = pd.concat([df_w, df_d], axis=1, ignore_index=False)
a.shape 
a
my_df['Transaction Category'].value_counts()
final_df = (pd.merge(my_df, a, on='Transaction Category', how='inner'))
final_df

df_w


,Withdrawal Percentage by Fiscal YTD
Transaction Category,
HHS - Grants to States for Medicaid,14.283693
HHS - Federal Supple Med Insr Trust Fund,12.611992
Dept of Defense (DoD) - misc,10.876775
HHS - Federal Hospital Insr Trust Fund,9.441275
Dept of Education (ED),5.395698
VA - Benefits,4.631366
HHS - Medicare Prescription Drugs,3.456073
HHS - Marketplace Payments,2.950448
USDA - Supp Nutrition Assist Prog (SNAP),2.530272


In [57]:

b = "output.csv"
final_df.to_csv(b, index=False)
final_df

,Record Date,Type of Account,Transaction Type,Transaction Category,Transactions Today,Transactions Month to Date,Transactions Fiscal Year to Date,Table Number,Table Name,Source Line Number,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number,"(Withdrawal Percentage by Fiscal YTD,)","(Deposit Percentage by Fiscal YTD,)"
0,2025-01-24,Treasury General Account (TGA),Withdrawals,NASA,35,1109,7452,II,Deposits and Withdrawals of Operating Cash,167,2025,2,2025,1,1,24,0.532418,NaN
1,2025-01-24,Treasury General Account (TGA),Deposits,Dept of Agriculture (USDA) - misc,19,270,1202,II,Deposits and Withdrawals of Operating Cash,1,2025,2,2025,1,1,24,0.754616,0.010240
2,2025-01-24,Treasury General Account (TGA),Deposits,USDA - Commodity Credit Corporation,47,573,2147,II,Deposits and Withdrawals of Operating Cash,2,2025,2,2025,1,1,24,0.585360,0.018291
3,2025-01-24,Treasury General Account (TGA),Deposits,USDA - Federal Crop Insurance Corp Fund,0,1,18,II,Deposits and Withdrawals of Operating Cash,3,2025,2,2025,1,1,24,0.822919,0.000153
4,2025-01-24,Treasury General Account (TGA),Deposits,USDA - Loan Repayments,13,422,2437,II,Deposits and Withdrawals of Operating Cash,4,2025,2,2025,1,1,24,NaN,0.020761
5,2025-01-24,Treasury General Account (TGA),Deposits,Dept of Commerce (DOC),17,753,2035,II,Deposits and Withdrawals of Operating Cash,5,2025,2,2025,1,1,24,0.561354,0.017336
6,2025-01-24,Treasury General Account (TGA),Deposits,Dept of Defense (DoD) - misc,52,1018,5440,II,Deposits and Withdrawals of Operating Cash,6,2025,2,2025,1,1,24,10.876775,0.046344
7,2025-01-24,Treasury General Account (TGA),Deposits,DoD - Health,18,107,695,II,Deposits and Withdrawals of Operating Cash,7,2025,2,2025,1,1,24,0.673953,0.005921
8,2025-01-24,Treasury General Account (TGA),Deposits,Dept of Education (ED),169,3942,17401,II,Deposits and Withdrawals of Operating Cash,8,2025,2,2025,1,1,24,5.395698,0.148241
9,2025-01-24,Treasury General Account (TGA),Deposits,Dept of Energy (DOE),25,392,2587,II,Deposits and Withdrawals of Operating Cash,9,2025,2,2025,1,1,24,1.441430,0.022039
